# Tutorial 08: modify a linear system on restriction

In this tutorial we solve the problem

$$\begin{cases}
-\Delta u = f, & \text{in } \Omega,\\
 u   = g, & \text{on } \Gamma = \partial\Omega,
\end{cases}$$

where $\Omega$ is the unit ball in 2D.

We compare the following two cases:
* **strong imposition of Dirichlet BCs**:
the corresponding weak formulation is
$$
\text{find } u \in V_g \text{ s.t. } \int_\Omega \nabla u \cdot \nabla v = \int_\Omega f v, \quad \forall v \in V_0\\
$$
where
$$
V_g = \{v \in H^1(\Omega): v|_\Gamma = g\},\\
V_0 = \{v \in H^1(\Omega): v|_\Gamma = 0\}.\\
$$
* **penalty imposition of Dirichlet BCs**: this requires first the discretization of the associated homogeneous Neumann problem
$$
\text{find } u \in V \text{ s.t. } \int_\Omega \nabla u \cdot \nabla v = \int_\Omega f v, \quad \forall v \in V\\
$$
where
$$
V = H^1(\Omega),
$$
which we rewrite in matrix form as
$$ \text{find } x \in \mathbb{R}^n \text{ s.t. } A x = b,$$
where $A \in \mathbb{R}^{n \times n}$ and $b \in \mathbb{R}^n$ are the left-hand side and right-hand side of the discrete Neumann problem.

In order to impose Dirichlet boundary conditions, we solve the modified system
$$ \text{find } \tilde{x} \in \mathbb{R}^n \text{ s.t. } \tilde{A} \tilde{x} = \tilde{b},$$
where
$$\tilde{A} = A + P, \qquad \tilde{b} = b + q.$$

The penalty matrix $P$ is defined as
$$
P_{ij} =
\begin{cases}
\mu, & \text{if $i = j$, and $i$ is a boundary DOF},\\
0, & \text{otherwise},
\end{cases}$$
and the penalty vector $q$ as
$$
q_i = 
\begin{cases}
\mu g(\mathbf{x}_i), & \text{if $i$ is a boundary DOF},\\
0, & \text{otherwise},
\end{cases}$$
where $\mu \in \mathbb{R}$ is (large) scalar number, and $\mathbf{x}_i$ denotes the coordinate of the entity associated at DOF $i$.

The preferred way to impose non-homogeneous Dirichlet boundary conditions should still
either be through `dirichletbc` objects (see tutorial 01) or lagrange multipliers
(see tutorial 03). This example is meant to show how to use the list of degrees of freedom
associated to a specific restriction to perform local modifications to assembled
matrices.

In [ ]:
import typing

In [ ]:
import dolfinx.fem
import dolfinx.io
import gmsh
import mpi4py
import numpy as np
import petsc4py
import ufl

In [ ]:
import multiphenicsx.fem
import multiphenicsx.io
import multiphenicsx.mesh

### Geometrical parameters

In [ ]:
r = 3
lcar = 1. / 4.

### Mesh

In [ ]:
gmsh.initialize()
gmsh.model.add("mesh")

In [ ]:
p0 = gmsh.model.geo.addPoint(0.0, 0.0, 0.0, lcar)
p1 = gmsh.model.geo.addPoint(0.0, +r, 0.0, lcar)
p2 = gmsh.model.geo.addPoint(0.0, -r, 0.0, lcar)
c0 = gmsh.model.geo.addCircleArc(p1, p0, p2)
c1 = gmsh.model.geo.addCircleArc(p2, p0, p1)
boundary = gmsh.model.geo.addCurveLoop([c0, c1])
domain = gmsh.model.geo.addPlaneSurface([boundary])

In [ ]:
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(1, [c0, c1], 1)
gmsh.model.addPhysicalGroup(2, [boundary], 0)
gmsh.model.mesh.generate(2)

In [ ]:
mesh, subdomains, boundaries = multiphenicsx.mesh.gmsh_to_fenicsx(gmsh.model, gdim=2)
gmsh.finalize()

In [ ]:
facets_Gamma = boundaries.indices[boundaries.values == 1]

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

In [ ]:
multiphenicsx.io.plot_mesh_entities(mesh, mesh.topology.dim - 1, facets_Gamma)

### Helper functions

In [ ]:
def generate_penalty_system(
    V: dolfinx.fem.FunctionSpace, restriction: np.typing.NDArray[np.int32], penalty: np.float64,
    g: dolfinx.fem.Function
) -> typing.Tuple[petsc4py.PETSc.Mat, petsc4py.PETSc.Vec]:
    """Generate matrix and vector to be added to the system to handle the penalty terms."""
    # Assemble penalty matrix.
    p = ufl.inner(u, v) * ufl.ds  # this form is not used per se, just to allocate the correct sparsity pattern
    P = dolfinx.fem.petsc.create_matrix(dolfinx.fem.form(p))  # this creates a zero matrix
    for dof in restriction:
        P.setValuesLocal([dof], [dof], [penalty])  # set P_{ij} = penalty * \delta_{ij}, if i on boundary
    P.assemble()
    # Next, assemble the penalty vector.
    q = dolfinx.cpp.la.petsc.create_vector(V.dofmap.index_map, V.dofmap.index_map_bs)
    with q.localForm() as q_local, g.vector.localForm() as g_local:
        q_local[restriction] = penalty * g_local[restriction]
    # Return matrix and vector
    return P, q

### Penalty imposition of Dirichlet BCs

In [ ]:
# Define a function space
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 2))

In [ ]:
# Define trial and test functions
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

In [ ]:
# Define problem forms
g = dolfinx.fem.Function(V)
g.interpolate(lambda x: np.sin(3 * x[0] + 1) * np.sin(3 * x[1] + 1))
a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
f = ufl.inner(1, v) * ufl.dx
a_cpp = dolfinx.fem.form(a)
f_cpp = dolfinx.fem.form(f)

In [ ]:
# Assemble the linear system
A = dolfinx.fem.petsc.assemble_matrix(a_cpp)
A.assemble()
F = dolfinx.fem.petsc.assemble_vector(f_cpp)
F.ghostUpdate(addv=petsc4py.PETSc.InsertMode.ADD, mode=petsc4py.PETSc.ScatterMode.REVERSE)

In [ ]:
# Add penalty terms
dofs_V_Gamma = dolfinx.fem.locate_dofs_topological(V, boundaries.dim, facets_Gamma)
(P, Q) = generate_penalty_system(V, dofs_V_Gamma, 1.e10, g)

In [ ]:
u = dolfinx.fem.Function(V)
ksp = petsc4py.PETSc.KSP()
ksp.create(mesh.comm)
ksp.setOperators(A + P)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.solve(F + Q, u.vector)
u.vector.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)

In [ ]:
multiphenicsx.io.plot_scalar_field(u, "u")

### Strong imposition of Dirichlet BCs

In [ ]:
# Define Dirichlet BC object on Gamma
bc_ex = dolfinx.fem.dirichletbc(g, dofs_V_Gamma)

In [ ]:
# Solve
u_ex = dolfinx.fem.Function(V)
problem_ex = dolfinx.fem.petsc.LinearProblem(
    a, f, bcs=[bc_ex], u=u_ex,
    petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
problem_ex.solve()
u_ex.vector.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)

In [ ]:
multiphenicsx.io.plot_scalar_field(u_ex, "u")

### Comparison and error computation

In [ ]:
u_ex_norm = np.sqrt(mesh.comm.allreduce(
    dolfinx.fem.assemble_scalar(dolfinx.fem.form(ufl.inner(ufl.grad(u_ex), ufl.grad(u_ex)) * ufl.dx)),
    op=mpi4py.MPI.SUM))
err_norm = np.sqrt(mesh.comm.allreduce(
    dolfinx.fem.assemble_scalar(dolfinx.fem.form(ufl.inner(ufl.grad(u_ex - u), ufl.grad(u_ex - u)) * ufl.dx)),
    op=mpi4py.MPI.SUM))
print("Relative error is equal to", err_norm / u_ex_norm)
assert np.isclose(err_norm / u_ex_norm, 0., atol=1.e-10)